# Python para ciencia e ingeniería. Clase 6/6

**Tallerista**: Martín Gaitán (Phasety)

**Colaboradores**: Jairo Trad (Insus) y Julián Scortechini (Phasety)

                                
                                Agosto/Septiembre de 2013


----
Licencia: ![](http://i.creativecommons.org/l/by-sa/2.5/ar/88x31.png)

# F2PY: Fortran ♥ Python, 


F2PY es un software (escrita en Python) que genera automáticamente el "pegamento" necesario para interfacear código Fortran y Python. 

Hay dos casos en los que nos puede venir bien usar F2PY.

* Queremos escribir una parte de cálculo intensivo en Fortran para que sea eficiente, pero queremos controlar la lógica del programa desde Python.
* Tenemos código legado y el esfuerzo de portarlo a otro lenguaje es demasiado grande (código ilegible, ausencia de comentarios, fragilidad). Aún podemos aprovecharlo: con F2PY podemos llamarlo desde Python.


F2py no es un compilador en sí mismo, sino que utiliza (casi) cualquier compilador Fortran subyacente. En vez de un archivo ejecutable (un ".exe" en Windows) genera una biblioteca dinámica importable desde Python como cualquier módulo. 

Nosotros vamos a utilizar el compilador **gfortran** que ya viene instalado con Anaconda. En linux basta hacer

      $ sudo apt-get install gfortran

###ATENTI: sólo usuarios de Windows/Anaconda

Antes de empezar vamos a cambiar la configuración de `gfortran` por defecto que trae Anaconda. Abrir el archivo `C:/Anaconda/Scripts/gfortran.bat` y modificarlo para que linkee estáticamente con las bibliotecas `libgcc` y `libgfortran`. Podemos hacerlo directamente **ejecutando la siguiente celda** que usa el magic ``%%file`` que escribe el contenido de la celda en un archivo

In [ ]:
%%file C:/Anaconda/Scripts/gfortran.bat
@echo off
%~f0\..\..\MinGW\bin\gfortran.exe -static-libgcc -static-libgfortran %*

## ¡Empecemos! 

Vamos a comenzar con un ejemplo del primer caso de uso: usar Fortran por cuestiones de performance. 

Si bien hay versiones más modernas, vamos a basarnos en el estándar **Fortran 90** porque es bien soportado por f2py y todos los compiladores, es muchísimo mejor que FORTRAN 77. Una buena y breve referencia es http://www.ews.uiuc.edu/~mrgates2/docs/fortran.html . El sitio http://fortran90.org/ tambien provee información valiosa y una **guia de estilo** sencilla que es bueno respetar en **todo código Fortran nuevo**. 

Pero antes, conozcamos el **Zen de Python**

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


¡El Zen de Python **no es sólo** para Python! **Aplicalo también para Fortran** (y para jugar al fútbol como el Barcelona). 

Especialmente el que dice *"Readability counts"*



In [1]:
%%file suma.f90 

subroutine dot(n, v)
    ! Compute the dot product between u and v (length n) and put the result in product
     
    integer, intent(in) :: n
    integer, intent(out) :: v
    v = n + 2
    
end subroutine

Overwriting suma.f90


Guardamos nuestro código Fortran en un módulo de código fuente llamado `dot_product.f90` Ahora lo compilamos usando f2py. El flag -m indica el nombre que se le dará a la biblioteca (que es como se importará dentro de de Python) y el flag -c los fuentes que debe compilar (en este caso sólo el que acabamos de crear). 

In [2]:
# Si queres evitar el output podés guardar en una variable
# _ = !f2py -m dot_product -c dot_product.f90   
!/home/tin/.virtualenvs/curso/bin/f2py3.4 -m suma -c suma.f90 

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "suma" sources
f2py options: []
f2py:> /tmp/tmph2h41h8s/src.linux-x86_64-3.4/sumamodule.c
creating /tmp/tmph2h41h8s/src.linux-x86_64-3.4
Reading fortran codes...
	Reading file 'suma.f90' (format:free)
Post-processing...
	Block: suma
			Block: dot
Post-processing (stage 2)...
Building modules...
	Building module "suma"...
		Constructing wrapper function "dot"...
		  v = dot(n)
	Wrote C/API module "suma" to file "/tmp/tmph2h41h8s/src.linux-x86_64-3.4/sumamodule.c"
  adding '/tmp/tmph2h41h8s/src.linux-x86_64-3.4/fortranobject.c' to sources.
  adding '/tmp/tmph2h41h8s/src.linux-x86_64-3.4' to include_dirs.
copying /home/tin/.virtualenvs/curso/lib/python3.4/site-packages/numpy/f2py/src/fortranobject.c -> /tmp/tmph2h41h8s/src.linu

¡Listo! Tendremos una biblioteca importable desde Python. En Widows será `dot_product.pyw` y en Linux `dot_product.so`



In [7]:
import suma

ImportError: No module named suma

F2py convierte **subrutinas y funciones** de Fortran en funciones de Python. Dentro del módulo `dot_product` tendremos una función llamada `dot` que no es más que  la subrutina que habiamos definido en Fortran. F2py le crea automáticamente algo de ayuda que podemos ver con el **?** o directamente imprimiendo la variable `__doc__` del objeto

In [7]:
import dot_product
dot_product.dot?

Como habiamos declarado que **p** era un valor de salida (`intent(out)`), F2py no nos pide que lo pasemos, sino que lo devuelve como resultado de la función. 

Pero hay más magia: **n** sólo se usa para dimensionar los arrays u y v. Entonces F2Py nos ahorra el trabajo de pasarlo y lo infiere por el tamaño de los arrays.

¿Y Funciona?

In [6]:
import numpy as np
u = np.array([1., 2., 0.])
v = np.array([2., .5, 0.])
dot_product.dot(u, v)

3.0

¡YEAHHHHHHH!

### Algo más pulentoso


Esta rutina calcula una **matriz de distancias** a partir de una matriz de m puntos n-dimensionales ordenados por filas


In [8]:
%%file distance.f90

subroutine euclidean(X,D,m,n)
  ! Computes the distance between m points using Euclidean distance (2-norm) 
  ! as the distance metric between the points. 
  ! The points are arranged as m n-dimensional row vectors in the matrix X.

  integer :: n,m
  double precision, intent(in) :: X(m,n)
  double precision, intent(out) :: D(m,m) 
  integer :: i,j,k
  double precision :: r 
  do i = 1,m 
      do j = 1,m 
          r = 0
          do k = 1,n 
              r = r + (X(i,k) - X(j,k)) * (X(i,k) - X(j,k)) 
          end do 
          D(i,j) = sqrt(r) 
      end do 
  end do 
end subroutine


Overwriting distance.f90


In [9]:
!/home/tin/.virtualenvs/curso/bin/f2py3.4 -m distance -c distance.f90   

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "distance" sources
f2py options: []
f2py:> /tmp/tmprr433we_/src.linux-x86_64-3.4/distancemodule.c
creating /tmp/tmprr433we_/src.linux-x86_64-3.4
Reading fortran codes...
	Reading file 'distance.f90' (format:free)
Post-processing...
	Block: distance
			Block: euclidean
Post-processing (stage 2)...
Building modules...
	Building module "distance"...
		Constructing wrapper function "euclidean"...
		  d = euclidean(x,[m,n])
	Wrote C/API module "distance" to file "/tmp/tmprr433we_/src.linux-x86_64-3.4/distancemodule.c"
  adding '/tmp/tmprr433we_/src.linux-x86_64-3.4/fortranobject.c' to sources.
  adding '/tmp/tmprr433we_/src.linux-x86_64-3.4' to include_dirs.
copying /home/tin/.virtualenvs/curso/lib/python3.4/site-packages/numpy/f

In [11]:
import distance
print(distance.euclidean.__doc__)

d = euclidean(x,[m,n])

Wrapper for ``euclidean``.

Parameters
----------
x : input rank-2 array('d') with bounds (m,n)

Other Parameters
----------------
m : input int, optional
    Default: shape(x,0)
n : input int, optional
    Default: shape(x,1)

Returns
-------
d : rank-2 array('d') with bounds (m,m)



In [13]:
import numpy as np
X = np.random.random((1000, 3))   # Mil puntos en R^3
%timeit distance.euclidean(X)

100 loops, best of 3: 8.89 ms per loop


Comparemos con la versión de este algoritmo que viene en Scipy

In [14]:
from scipy.spatial.distance import cdist
%timeit cdist(X, X)

100 loops, best of 3: 7.65 ms per loop


Para los desconfiados, acá la prueba de que es lo mismo:

In [15]:
np.all(cdist(X, X) == distance.euclidean(X))

True

In [1]:
%install_ext https://raw.github.com/mgaitan/fortran_magic/master/fortranmagic.py

Installed fortranmagic.py. To use it, type:
  %load_ext fortranmagic


In [2]:
%load_ext fortranmagic

In [3]:
%%fortran

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1

In [4]:
f1(4, 2)

-0.279415488243103

Ok, probablemente los de Scipy programan mejor que nosotros :)

Un algoritmo equivalente implementado en "numpy pelado" lo puse acá https://gist.github.com/mgaitan/6465190  

## Un caso real: código "científico" 

Este código es una versión simplificada que calcula parámetros de EOS cúbicas SRK o PR a partir de las constantes de un compuesto Tc, Pc, y $\omega$

In [ ]:
%%file CubicParam_simpler.for

	program ModelsParam
      implicit DOUBLE PRECISION (A-H,O-Z)
      PARAMETER (RGAS=0.08314472d0)
C	Critical constants must be given in K and bar
C	b will be in L/mol and ac in bar*(L/mol)**2
      PARAMETER (A0=0.0017,B0=1.9681,C0=-2.7238)
      PARAMETER (A1=-2.4407,B1=7.4513,C1=12.504)
	dimension D(6)
	COMMON /Tcdc/ Tc,dc
	COMMON /ABd1/ a,b,del1
	D=[0.428363,18.496215,0.338426,0.660,789.723105,2.512392]
	NC=2
	NIN=1
	nout=9
      OPEN(NIN,FILE='CONPARIN.DAT')
      OPEN(NOUT,FILE='CONPAROUT.DAT')
	read(NIN,*)ICALC,NMODEL
	IF(NMODEL.EQ.4)GO TO 104
    READ(NIN,*)Tc,Pc,OM
    RT=RGAS*Tc
	END IF
	IF(nmodel.EQ.1)THEN
C		SRK EOS
		del1=1.0D0
	ELSE
C		PR EOS
		del1=1.0D0+sqrt(2.0)
	END IF
11	d1=(1+del1**2)/(1+del1)
	y=1+(2*(1+del1))**(1.0d0/3)+(4/(1+del1))**(1.0d0/3)
	OMa=(3*y*y+3*y*d1+d1**2+d1-1.0d0)/(3*y+d1-1.0d0)**2
	OMb=1/(3*y+d1-1.0d0)
	Zc=y/(3*y+d1-1.0d0)

	ac=OMa*RT**2/Pc
	b=OMb*RT/Pc

	Zc=y/(3*y+d1-1.0d0)
	Vceos=Zc*RGAS*Tc/Pc
	IF(nmodel.eq.1)THEN
		rm=0.48+1.574*OM-0.175*OM**2  ! m from SRK
	ELSE
		rm=0.37464+1.54226*OM-0.26992*OM**2  ! m from PR
	END IF
	write(nout,1)Tc,Pc,OM,Vceos  ! it used to be "Vceos,OM" (changed 5/7/13 to imitate gpecin standard)
	write(nout,2)ac,b,rm
c	write(NOUT,*)
c	write(nout,*)'Tc, Pc and Vc are given in K, bar and L/mol respectively'
 1	FORMAT(F10.4,F10.4,F10.6,F9.5)
 2	FORMAT(F10.4,F10.6,F10.6,F10.5)
104	continue
	close(unit=NIN)
	close(unit=nout)
	end

Este código es más parecido a Fortran 77 que a Fortran 90, pero **no compilaría** en un compilador de f77 estricto. Y ¿me creen si les digo que el original es muchísimo más intrincado?

Un archivo de entrada "CONPARIN.DAT" tiene la forma

In [ ]:
%%file CONPARIN.DAT
0  1
 369.83  42.48  0.152291
 

El problema con esta arquitectura de entrada/salida es que es muy frágil: qué significa cada número está dado por su posición, que puede cambiar a lo largo del tiempo o significar distintas cosas en función de otro número, etc. Hay que ser un poco adivino o demasiado "científico" para leer y **mantener** este código.
 
Pero probemos que funciona

In [ ]:
!gfortran -o cubic_param CubicParam_simpler.for

In [ ]:
!./cubic_param   # ./cubic_param  en linux

In [ ]:
%pycat CONPAROUT.DAT

Si convirtieramos con F2py **no tendria ninguna función** y es poque sólo hay un gran bloque `program` 

In [ ]:
_ = !f2py -m cubic_param -c CubicParam_simpler.for
_[-2:]

In [ ]:
import cubic_param

Sólo tenemos los bloques **commons**

In [ ]:
dir(cubic_param)

Reescribiremos el módulo Fortran un poquito para convertirlo en una subrutina, que reciba los parámetros en vez de leerlos desde un archivo. De paso, lo convertiremos (un poco) a **Fortran 90** y simplificando porque ya sabemos: **la legibilidad cuenta**. 

Algunas reglas (no exhaustivas):

- **Sólo** usar mayúsculas para definir constantes/parámetros
- Usá **nombres de variables significativos**: no funciona más rápido por ser críptico. 
- De nuevo: **la legibilidad es lo más importante**. Sé explícito y poné comentarios cuando el código no es obvio
- Dejá **sangrías** (de 4 espacios) para destacar las estructuras de control (como si fuera python)
- Usá los operadores lógicos **==, >=, /=, <**, en vez de `.EQ., .GE. ... ` . Son "menos ruidosos".
- Si tenés que continuar una linea en la siguiente, hacelo con `&` al final 
- Los comentarios empezalos siempre con **!**  y en el nivel de sangría en el que estás.
- Declará el **intent** de tus parámetros. Explícito es mejor que implícito.
- Evitá, por el amor de Messi, el uso de `goto`
- Hacé subrutinas lo más pequeñas posibles que hagan **una** cosa, bien.
- Tratá de no usar **parámetros que cambien el significado** de entrada/salida. Casos del tipo *"si primero le pasas un 0 lo que sigue es la temperatura y lo que devuelve es el volumen, pero si le pasas un 1 lo que espera es la cantidad de goles de Messi en la selección y devuelve la distancia Tierra-Marte de mañana. Si la pasás un 3 algo hace pero no me acuerdo qué"*. Si no lo podés evitar, definí constantes explícitamente y usalas en los "if" en vez de los números literales. Un "...pero es casi igual" es síntoma de que tenés que separar código en rutinas auxiliares.
- Los comentarios empezalos siempre con **!**  y en el nivel de sangría en el que estás.
  Si son rutinas auxiliares poneles un guión bajo como prefijo  (`_calc_something(...)`)
- Los principios [KISS](http://en.wikipedia.org/wiki/KISS_principle) y [DRY](http://en.wikipedia.org/wiki/DRY_principle) son nuestros dogmas. Mantenelo simple y ¡no repitas código! Si hay una subrutina que usas seguido separala en un nuevo módulo e incluílo en la compilación. El **Copy & paste es malo**, malo, malo. 
- Si no podés evitar **leer datos** desde un archivo, **hacelo en una sóla rutina bien documentada** (con ejemplos de input) y no con `read(NIN, *)` desparramados por doquier. Lo mismo aplica a la rutina de "escritura" del output. 
- El código comentado sólo mete ruido. Un "puede servir mañana", es sintoma de que necesitas usar un sistema de control de versiones. 
- Comentarios tipo "cambié esto el 1 de agosto porque necesitaba..." es sintoma de que necesitas usar un sistema de control de versiones. 
- Necesitas usar un sistema de control de versiones. . Dropbox no lo es. 
- ...

Nota: El script [f2f.pl](https://bitbucket.org/lemonlab/f2f/) puede ayudar un poco automatizando algunas cosas de la migración.

Bueno, basta de adoctrinamiento. 

In [ ]:
%%file CubicParam_simpler.f90

subroutine modelsparam(nmodel, tc, pc, om, constants, parameters)

    implicit double precision (a-h,o-z)
    parameter (rgas=0.08314472d0)

    !  Critical constants must be given in k and bar
    !  b will be in l/mol and ac in bar*(l/mol)**2

    parameter (a0=0.0017, b0=1.9681, c0=-2.7238)
    parameter (a1=-2.4407, b1=7.4513, c1=12.504)
    dimension d(6), constants(4), parameters(3)

    !*** Atenti a la linea que sigue *******
    !f2py intent(out) :: constants, parameters

    d = [0.428363, 18.496215, 0.338426, 0.660, 789.723105, 2.512392]
    rt = rgas*tc

    if(nmodel == 1) then
        ! srk eos
        del1 = 1.0d0
    else
        ! pr eos
        del1 = 1.0d0 + sqrt(2.0)
    end if

    d1 = (1+del1**2)/(1+del1)
    y = 1+(2*(1+del1))**(1.0d0/3)+(4/(1+del1))**(1.0d0/3)
    oma = (3*y*y+3*y*d1+d1**2+d1-1.0d0)/(3*y+d1-1.0d0)**2
    omb = 1/(3*y+d1-1.0d0)
    zc = y/(3*y+d1-1.0d0)

    ac = oma*rt**2/pc
    b = omb*rt/pc

    zc = y/(3*y+d1-1.0d0)
    vceos=zc*rgas*tc/pc

    if(nmodel == 1) then
        rm = 0.48+1.574*om-0.175*om**2  ! m from srk
    else
        rm = 0.37464+1.54226*om-0.26992*om**2  ! m from pr
    end if

    constants = (/tc, pc, om, vceos /)
    parameters = (/ac, b, rm/)

end subroutine 

In [ ]:
_ = !f2py -m cubic_param_new -c CubicParam_simpler.f90

In [ ]:
import cubic_param_new
print cubic_param_new.modelsparam.__doc__

In [ ]:
constants, parameters = cubic_param_new.modelsparam(1, tc=369.83, pc=42.48, om=0.152291)

In [ ]:
print constants
print parameters

In [ ]:
%pycat CONPAROUT.DAT

¡Genial! 

### ¿Cómo seguimos?


F2Py es **muy poderoso** pero hay que ayudarlo. 

Un **patrón típico** es el de cálculos iterativos que dan "distinta cantidad de output". En nuestro ejemplo siempre se devolvian dos arrays de longitud conocida, pero **¿que pasa si no sabemos de antemano el tamaño de los arrays de salida?** 

Escribiendo archivos de texto a medida que se encuentran los puntos se evita "almacenar" los valores en un array, pero ya sabemos que eso es una mala arquitectura porque a) **escribir al disco es lento** (si fortran es rápido, ahí perdemos la ventaja) y b) casi siempre queremos los resultados para hacer nuevos cómputos: otras cuentas, gráficos, o mostrarlos en una bonita interfaz, etc. Nada de eso es más fácil desde un archivo de texto. 


Una forma simple de resolver eso es dimensionar el array contenedor bien grande y llevar un contador (entero) de datos válidos. Tanto el array como el contador se devuelven


In [1]:
%%file output_variable.f90

subroutine algo(nin, output, nout)
    implicit none
    integer, intent(in) :: nin
    real, dimension(50000), intent(out) :: output
    integer, intent(out) :: nout
    integer :: i
            
    nout = 0
    ! hacer calculos que van llenando
    do i=1,2*nin
        output(i) = i ** .5
        nout = nout + 1
    end do
end subroutine

Writing output_variable.f90


In [2]:
_ = !f2py -m variable -c output_variable.f90

In [3]:
import variable

In [4]:
output, validos = variable.algo(10)
output.size, validos

(50000, 20)

Entonces lo mejor es hacer una función (o método) que **envuelva** a la de fortran y haga el postprocesado (y el preprocesado si hiciera falta), **mejorando de paso la documentación**. Esto se llama un **wrapper** 

In [5]:
def raizde2(cantidad):
    """
    dado un entero, devuelve un vector de el doble de tamaño 
    donde cada elemento i tiene valor sqrt(i + 1)  (raiz cuadrada)
    """ 
    output, validos = variable.algo(cantidad)
    return output[:validos]

In [6]:
raizde2(10)[1]

1.4142135

In [7]:
raizde2(10).size

20

In [8]:
help(raizde2)

Help on function raizde2 in module __main__:

raizde2(cantidad)
    dado un entero, devuelve un vector de el doble de tamaño 
    donde cada elemento i tiene valor sqrt(i + 1)  (raiz cuadrada)



### ¿Un deadmatch ?

In [9]:
import numpy as np
def raizde2_np(cantidad):
    """
    dado un entero, devuelve un vector de el doble de tamaño 
    donde cada elemento i tiene valor sqrt(i + 1)  (raiz cuadrada)
    """ 
    return np.sqrt(np.arange(1, 2*cantidad + 1, dtype=np.float32))

In [10]:
(raizde2(10) == raizde2_np(10)).all()

True

In [11]:
%timeit raizde2(10000)

1000 loops, best of 3: 226 µs per loop


In [12]:
%timeit raizde2_np(10000)

The slowest run took 4.68 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 34.6 µs per loop


In [14]:
226/34.

6.647058823529412

numpy es un 664% más rápido para este caso sencillo ¡En tu cara compilador!